# Latent Space Interpolation Demo (MNIST digit)
- In this notebook, we will load a pre-trained LaDDer model for MNIST digit dataset. 
- We will then interpolate between a pair of MNIST images over k=5 steps, using the following two methods:
  - shortest path (SP) method, which linearly interpolates between the two images in the latent space
  - shortest likelihood path (SLP) method (**our proposal**), which considers the derived data manifold in the latent space and only interpolates samples with high likelihood.  
- We will visualise the interpolated images in the 2D latent space between the two methods to show the difference in the interpolated paths. 

### Here you select some hyper-parameters for the interpolation task
- initialised_method: we initialise the path as a linear interpolation 
- n_step: the number of interpolated images to be generated

In [ ]:
initialise_method = 'straight_line'
n_step = 8

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import random

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, ion, show, savefig, cla, figure
from matplotlib.patches import Ellipse
import matplotlib._color_data as mcd
matplotlib.rcParams.update({'font.size': 16})

from sklearn.mixture import GaussianMixture
from scipy.stats import norm, multivariate_normal
# %matplotlib notebook

In [ ]:
print(tf.__version__)

In [ ]:
# import sys
# sys.path.append("..")

from codes.utils import process_config, create_dirs, get_args, count_trainable_variables
from codes.data_loader import DataGenerator
from codes.models import MNISTModel_digit, MNISTModel_fashion, CelebAModel_densenet
from codes.trainers import MNISTTrainer_joint_training, CelebATrainer_joint_training
from demo.demo_tools import *

This interpolation demo does **not** require GPU to run.

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print("Available device is {}".format(get_available_gpus()))

## Load the pre-trained model
We provided pre-trained models for all 3 datasets in `../pretrained_models/` folder.

Please specify the dataset name (in the variable `exp_name` below) of which you want to load the corresponding model.  

In [ ]:
# specify the dataset that you want to interpolate
# the current code only works with mnist datasets.
# the demo for celebA dataset will be created in another demo. 
exp_name = 'mnist_digit' # 'mnist_fashion' or 'celebA'

In [ ]:
config = process_config('demo/{}_config.json'.format(exp_name))

# create the experiments dirs
create_dirs([config['result_dir'], config['checkpoint_dir']])

# create tensorflow session
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True
sess = tf.Session(config=sess_config)

# create your data generator
data = DataGenerator(config, sess)

In [ ]:
# build the laDDer model
if config['exp_name'] == "mnist_digit":
    model = MNISTModel_digit(config)
elif config['exp_name'] == "mnist_fashion":
    model = MNISTModel_fashion(config)
elif config['exp_name'] == "celeba":
    model = CelebAModel_densenet(config)

# create a trainer
if config['exp_name'] in ["mnist_digit", "mnist_fashion"]:
    trainer = MNISTTrainer_joint_training(sess, model, data, config)
    sess.run(model.iterator.initializer,
             feed_dict={model.original_signal: data.train_set['image'],
                        model.seed: trainer.cur_epoch})
elif config['exp_name'] == "celeba":
    trainer = CelebATrainer_joint_training(sess, model, data, config)


In [ ]:
# load model if exists
model.load(sess, model="VAE")
if config['prior'] in ["ours", 'hierarchical', 'vampPrior']:
    model.load(sess, model="prior")

### Here we fit a GMM hyper prior for our model using 20k training images.

In [ ]:
# fit a GMM hyper prior 
if config['prior'] == "ours":
    samples = trainer.fit_GMM_VI(iterator=model.input_image, mode="accurate", space="t")
    if config['representation_size'] == 2:
        trainer.plot_prior_distribution(samples, mode="accurate-GM", style='circle', show_img=True)
        trainer.plot_prior_distribution(samples, mode="accurate-GM", style='density', show_img=True)
elif config['prior'] == "GMM":
    samples = trainer.fit_GMM_VI(iterator=model.input_image, mode="accurate", space="z")
    if config['code_size'] == 2:
        trainer.plot_prior_distribution(samples, mode="accurate-GM", style='circle', show_img=True)
        trainer.plot_prior_distribution(samples, mode="accurate-GM", style='density', show_img=True)

if config['prior'] in ['ours', 'GMM']:
    GM = dict()
    GM['m'] = (trainer.GM_prior_final.means_).astype(np.float32)
    GM['K'] = (trainer.GM_prior_final.covariances_).astype(np.float32)
    GM['w'] = (trainer.GM_prior_final.weights_).astype(np.float32)
else:
    GM = None

prior = define_prior_distribution(config, sess, model, gmm_info=GM)

### Here you select the two images that you want to interpolate between.

In [ ]:
# Get two query images - start and end of interpolation 
# please feel free to use different indices to try out different interpolation result 
save_dir = config['result_dir']

idx_start = 0
print("Start sample:")
embedding_start = get_embeddings_from_val_set(idx_start, config, exp_name, sess, data, model, trainer, 
                                              save_plot=False)

idx_end = 32
print("Target sample:")
embedding_end = get_embeddings_from_val_set(idx_end,  config, exp_name, sess, data, model, trainer, 
                                            save_plot=False)

## Our shortest likelihood path interpolation starts here.
- We first initialise the path using a linear path.
- We then define the interpolation objective as shown in Eq (9) in our paper.
- We then use AdamOptimizer to optimise the objective for 500 iterations. 
- The optimised embeddings correspond to the interpolated images. 

In [ ]:
# Define an optimisation objective 
# Generate a set of random embeddings as initialisation
if initialise_method == 'random':
    initialised_embeddings = generate_prior_embeddings(prior, n_embeddings=n_step)
else: 
    initialised_embeddings = np.linspace(embedding_start, embedding_end, n_step+1, endpoint=False)[1:]
# print(initialised_embeddings.shape)
# print(initialised_embeddings[0])

# Define tf.Variables for the 5 points in the intermediate steps
with tf.variable_scope('interpolation'):
    pts = tf.Variable(initialised_embeddings, dtype=tf.float32, trainable=True, name='intermediate_pts')

# Compute the path length (Euclidean)
pts_with_start = tf.concat([tf.expand_dims(embedding_start, 0), pts], axis=0)
pts_with_end = tf.concat([pts, tf.expand_dims(embedding_end, 0)], axis=0)
segment_length = tf.sqrt(tf.reduce_sum(tf.square(pts_with_end - pts_with_start), axis=1))
equal_length_constraint = tf.math.reduce_std(segment_length)
entire_path_length = tf.reduce_sum(segment_length)

# Evaluate the likelihood along the path
neg_ll = - tf.reduce_sum(prior.log_prob(pts))

w_equal_length = tf.placeholder(tf.float32, [])
w_path_dist = tf.placeholder(tf.float32, [])

# Overall objective
interpolation_obj = w_path_dist * entire_path_length + w_equal_length * equal_length_constraint + neg_ll

In [ ]:
w_step_size = 100
w_shortest_path = 10

# Define optimisation step
interpolation_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "interpolation")
num_interpolation = count_trainable_variables('interpolation')

lr_interpolation = tf.placeholder(tf.float32, [])
opt_interpolation = tf.train.AdamOptimizer(learning_rate=lr_interpolation,
                                beta1=0.9,
                                beta2=0.95)
sess.run(tf.variables_initializer(opt_interpolation.variables()))
gvs_interpolation = opt_interpolation.compute_gradients(interpolation_obj, var_list=interpolation_vars)
print('gvs interpolation: {}'.format(gvs_interpolation))
capped_gvs_interpolation = [(model.ClipIfNotNone(grad), var) for grad, var in gvs_interpolation]

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    update_step_interpolation = opt_interpolation.apply_gradients(capped_gvs_interpolation)

In [ ]:
# Generate a set of random embeddings as initialisation
if initialise_method == 'random':
    initialised_embeddings = generate_prior_embeddings(prior, n_embeddings=n_step)
pts.assign(initialised_embeddings)

sess.run(pts.initializer)
sess.run(tf.variables_initializer(opt_interpolation.variables()))

In [ ]:
# Optimisation over certain iterations
n_iter = 500
cur_lr = 1e-2
n_iter_print = 100

loss = []
pts_record = []
step_var_record = []
path_length_record = []
neg_ll_record = []
feed_dict = {lr_interpolation: cur_lr,
             w_equal_length: w_step_size,
             w_path_dist: w_shortest_path,
             model.original_signal: trainer.test_batch}
for i in range(n_iter):
    cur_loss, cur_pts, cur_step_var, cur_path_length, cur_ll, _ = sess.run([interpolation_obj,
                                     pts,
                                     equal_length_constraint,
                                     entire_path_length,
                                     neg_ll,
                                     update_step_interpolation], feed_dict=feed_dict)
    loss.append(cur_loss)
    pts_record.append(cur_pts)
    step_var_record.append(cur_step_var)
    path_length_record.append(cur_path_length)
    neg_ll_record.append(cur_ll)
    if i % n_iter_print == n_iter_print-1:
        print("Iter {}/{}: cur loss is {}".format(i+1, n_iter, cur_loss))
print("Step variance is {}".format(cur_step_var))

print("Losses in our interpolation objective are optimised as below:")
plot_interpolation_losses(loss, path_length_record, step_var_record, neg_ll_record, n_iter,
                          idx_start, idx_end, n_step, config)

## Here we demonstrate the interpolation results
- We first show the trace of interpolated images under our method.
- We then visualise the interpolated path in the 2D latent space. This is compared with the linear path using shortest path (SP) method. 

In [ ]:
# plot the interpolated images from start image to target image
print("SLP interpolation (ours)")
interpolated_embeddings = np.concatenate(
    [np.expand_dims(embedding_start, 0), cur_pts, np.expand_dims(embedding_end, 0)], axis=0)
plot_interpolated_images(interpolated_embeddings, config, exp_name, sess, data, model, trainer,
                         n_step, idx_start, idx_end, save_plot=True, name_input='SLP')

print("SP interpolation")
interpolated_embeddings = np.concatenate(
    [np.expand_dims(embedding_start, 0), initialised_embeddings, np.expand_dims(embedding_end, 0)], axis=0)
plot_interpolated_images(interpolated_embeddings, config, exp_name, sess, data, model, trainer,
                         n_step, idx_start, idx_end, save_plot=True, name_input='SP')

In [ ]:
# plot the interplated path in the 2D latent space for our SLP method
plot_optimised_path(
    cur_pts, config, GM, sess, model, trainer,
    embedding_start, embedding_end, idx_start, idx_end, n_step, 
    prior=prior, 
    plot_prior='density', 
    w=2., 
    save_plot=True, 
    grid_size=7, 
    font_size=16,
    name_input='SLP')

In [ ]:
# plot the interplated path in the 2D latent space for the linear SP method
plot_optimised_path(
    initialised_embeddings, config, GM, sess, model, trainer,
    embedding_start, embedding_end, idx_start, idx_end, n_step, 
    prior=prior, 
    plot_prior='density', 
    w=2., 
    save_plot=True, 
    grid_size=7., 
    font_size=16, 
    name_input='SP', 
    c='lightblue')